In [4]:
#using this to predict the intent!
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import pandas as pd

# Example data
data = [
    {"query": "Add a new student with name 'Alice', age 14, grade 9, section 'A'", "intent": "INSERT"},
    {"query": "Update attendance for student with id 3 to 95", "intent": "UPDATE"},
    {"query": "Get the names of all students in grade 10", "intent": "SELECT"},
    # Add more examples
]
df=pd.read_csv("studentdatset.csv")
# Prepare data
queries = [item["query"] for item in data]
intents = [item["intent"] for item in data]
queries=df["Query"]
intents=df["Intent"]
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(intents)

# Split data
X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

# Create and train model
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(X_train, y_train)

# Predict intent
def predict_intent(query):
    intent_idx = model.predict([query])[0]
    return label_encoder.inverse_transform([intent_idx])[0]

# Example prediction
print(predict_intent("Get the names of all students in grade 10"))


SELECT


In [48]:
from sklearn.multioutput import MultiOutputClassifier

# Example data with columns
data_with_columns = [
    {"query": "Add a new student with name 'Alice', age 14, grade 9, section 'A'", "columns": ["name", "age", "grade", "section"]},
    {"query": "Update attendance for student with id 3 to 95", "columns": ["attendance"]},
    {"query": "Get the names of all students in grade 10", "columns": ["name"]},
    # Add more examples
]

# Prepare data
queries = [item["query"] for item in data_with_columns]
columns = [item["columns"] for item in data_with_columns]
queries=df["Query"]
intents=df["Columns"]
# Binarize column labels


In [50]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(columns)
print(y)

[[1 0 1 1 1]
 [0 1 0 0 0]
 [0 0 0 1 0]]


In [ ]:

# Split data
X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

# Create and train model
column_model = make_pipeline(TfidfVectorizer(), MultiOutputClassifier(LogisticRegression()))
column_model.fit(X_train, y_train)

# Predict columns
def predict_columns(query):
    column_indices = column_model.predict([query])[0]
    return mlb.inverse_transform([column_indices])[0]

# Example prediction
print(predict_columns("Add a new student with name 'Alice', age 14, grade 9, section 'A'"))


In [42]:
def clean(word):
    word=str(word)
    if word=="*":
        return ['*']
    elif word=="nan":
        return []
    
    gg=word.replace("(","")
    gg=gg.replace(")","")
    ha =gg.split(", ")
    return ha
clean("name, grade")

['name', 'grade']

In [45]:
df=pd.read_csv("studentdatset.csv")

df.head()
df['Columns']=df['Columns'].apply(clean)


In [46]:
df.head()

,Query,Intent,Columns,Conditions
0,"Add a new student with name 'Alice', age 14, g...",INSERT,"[name, age, grade, section]","('Alice', 14, 9, 'A')"
1,Update attendance for student with id 3 to 95,UPDATE,[attendance],id = 3
2,Get the names of all students in grade 10,SELECT,[name],grade = 10
3,Delete student with id 5,DELETE,[],id = 5
4,"Add a new student with name 'Bob', age 15, gra...",INSERT,"[name, age, grade, section]","('Bob', 15, 10, 'B')"
